## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler,StringIndexer,VectorIndexer

In [4]:
sqlContext = SQLContext(sc)
df = sqlContext.read.load('/FileStore/tables/daily_weather.csv',format = 'com.databricks.spark.csv',header='true',inferSchema='true')

In [5]:
df.columns

Out[ 4 ]: 
['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [6]:
featureColumns = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        'rain_duration_9am']

In [7]:
df = df.drop('number')

In [8]:
df = df.na.drop()

In [9]:
df.count(),len(df.columns)

Out[ 8 ]: (1064, 10)

In [10]:
binarizer = Binarizer(threshold=24.99999,inputCol="relative_humidity_3pm",outputCol="label")
binarizedDF = binarizer.transform(df)

In [11]:
binarizedDF.describe()

Out[ 10 ]: DataFrame[summary: string, air_pressure_9am: string, air_temp_9am: string, avg_wind_direction_9am: string, avg_wind_speed_9am: string, max_wind_direction_9am: string, max_wind_speed_9am: string, rain_accumulation_9am: string, rain_duration_9am: string, relative_humidity_9am: string, relative_humidity_3pm: string, label: string]

In [12]:
binarizedDF.select("relative_humidity_3pm","label").show(4)

+---------------------+-----+
relative_humidity_3pm|label|
+---------------------+-----+
 36.160000000000494| 1.0|
 19.4265967985621| 0.0|
 14.460000000000045| 0.0|
 12.742547353761848| 0.0|
+---------------------+-----+
only showing top 4 rows

In [13]:
assembler = VectorAssembler(inputCols=featureColumns,outputCol="features")
assembled = assembler.transform(binarizedDF)

In [14]:
assembled.printSchema()

root
-- air_pressure_9am: double (nullable = true)
-- air_temp_9am: double (nullable = true)
-- avg_wind_direction_9am: double (nullable = true)
-- avg_wind_speed_9am: double (nullable = true)
-- max_wind_direction_9am: double (nullable = true)
-- max_wind_speed_9am: double (nullable = true)
-- rain_accumulation_9am: double (nullable = true)
-- rain_duration_9am: double (nullable = true)
-- relative_humidity_9am: double (nullable = true)
-- relative_humidity_3pm: double (nullable = true)
-- label: double (nullable = true)
-- features: vector (nullable = true)

In [15]:
(trainingData,testData) = assembled.randomSplit([0.8,0.2],seed = 13234)

In [16]:
trainingData.count(),testData.count()

Out[ 15 ]: (859, 205)

In [17]:
dt = DecisionTreeClassifier(labelCol="label",featuresCol="features",maxDepth=5,minInstancesPerNode=20,impurity="gini")

In [18]:
pipeline = Pipeline(stages=[dt])
model = pipeline.fit(trainingData)

In [19]:
predictions = model.transform(testData)

In [20]:
predictions.select("prediction","label").show(20)

+----------+-----+
prediction|label|
+----------+-----+
 1.0| 1.0|
 0.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 0.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 0.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 0.0| 0.0|
 1.0| 1.0|
+----------+-----+
only showing top 20 rows

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3895203204840548> in <module> () 
 ----> 1 predictions . select ( "prediction" , "label" ) . write . save ( path = "/FileStore/tables/predictions.csv" , format = "com.databricks.spark.csv" , header = 'true' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 734 self . _jwrite . save ( ) 
 735 else : 
 --> 736 self . _jwrite . save ( path ) 
 737 
 738 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'path dbfs:/FileStore/tables/predictions.csv already exists.;'

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [23]:
df.show(10)

+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
 air_pressure_9am| air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
918.0600000000087| 74.82200000000041| 271.1| 2.080354199999768| 295.39999999999986| 2.863283199999908| 0.0| 0.0| 42.42000000000046| 36.160000000000494|
917.3476881177097| 71.40384263106537| 101.93517935618371|2.4430092157340217| 140.47154847112498|3.5333236016106238| 0.0| 0.0| 24.328697291802207| 19.4265967985621|
923.0400000000084|60.637999999998776| 51.000000000000036|17.067852199999727| 63.6999999999999|22.100967200000003| 0.0| 20.0| 8.900000000000047| 14.460000000000045|
920.5027511759195| 70.13889486830053| 198.83213271746368| 4.337363056162703| 211.20334124325618| 5.190045360219378| 0.0| 0.0| 12.189101868764444| 12.742547353761848|
921.1600000000036| 44.29400000000284| 277.8|1.8566601999998955| 136.49999999999991| 2.863283199999954| 8.899999999998613| 14730.0| 92.41000000000044| 76.74000000000046|
 915.300000000006| 78.40400000000422| 182.8| 9.93201359999979| 188.99999999999986|10.983375399999995| 0.02000000000043656| 170.0| 35.130000000000265| 33.930000000000256|
915.5988675132801| 70.04330431686239| 177.87540718659943|3.7455865377244795| 186.60669587586753| 4.589632428699432| 0.0| 0.0| 10.657421657822635| 21.385656725200974|
918.0700000000023|51.710000000002346| 242.4| 2.527742199999939| 271.5999999999999|3.6462121999999924| 0.0| 0.0| 80.47000000000041| 74.92000000000041|
 920.080000000006| 80.58200000000384| 40.7| 4.518618799999764| 62.99999999999986| 5.883152199999977| 0.0| 0.0| 29.58000000000045| 24.030000000000427|
915.0100000000116| 47.4979999999998| 163.10000000000005| 4.943637399999545| 195.89999999999992| 6.576603599999951| 0.0| 0.0| 88.6000000000012| 68.0500000000012|
+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
only showing top 10 rows

In [24]:
predictions = sqlContext.read.load('/FileStore/tables/predictions.csv/part-00000-tid-7491931305090165215-8c5321c1-54a4-4785-aeae-2ed1d057cb4e-88-c000.csv',format = 'com.databricks.spark.csv',header='true',inferSchema='true')

In [25]:
evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")

In [26]:
accuracy = evaluator.evaluate(predictions)

In [27]:
predictions.show(10)

+----------+-----+
prediction|label|
+----------+-----+
 1.0| 1.0|
 0.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 1.0| 1.0|
 0.0| 1.0|
 1.0| 1.0|
+----------+-----+
only showing top 10 rows

In [28]:
print("Accuracy = %g " % (accuracy))

Accuracy = 0.795122

In [29]:
predictions.rdd.take(2)

Out[ 39 ]: [Row(prediction=1.0, label=1.0), Row(prediction=0.0, label=1.0)]

In [30]:
predictions.rdd.map(tuple).take(2)

Out[ 40 ]: [(1.0, 1.0), (0.0, 1.0)]

In [31]:
metrics = MulticlassMetrics(predictions.rdd.map(tuple))

In [32]:
metrics.confusionMatrix().toArray().transpose()

Out[ 44 ]: 
array([[ 85., 19.],
 [ 23., 78.]])

In [33]:
print ("Error = %g " % (1.0 - accuracy))

Error = 0.204878

In [34]:
print ("Accuracy = %.2g" % (accuracy * 100))

Accuracy = 80

In [35]:
metrics.confusionMatrix().toArray()

Out[ 51 ]: 
array([[ 85., 23.],
 [ 19., 78.]])